In [ ]:
!pip install langchain einops accelerate transformers bitsandbytes

In [1]:
from langchain import HuggingFacePipeline
from langchain import PromptTemplate,  LLMChain
from transformers import AutoTokenizer, AutoModelForCausalLM
import transformers
import os
import torch

In [2]:
# Check if cuda is available
torch.cuda.is_available()

True

In [5]:

# Define Model ID
model_id = "tiiuae/falcon-7b-instruct"
# Load Tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_id) # get the tokenizer of pretrained model falcon with 7b parameter
# Load Model
model = AutoModelForCausalLM.from_pretrained(model_id, # get the pretrained model falcon with 7b parameter
                                            cache_dir='./workspace/', # where to store data cache
                                            torch_dtype=torch.bfloat16, # compatible data type of model
                                            trust_remote_code=True,
                                            device_map="auto",
                                          offload_folder="offload" #  Whether or not to force the (re-)download of the model weights and configuration files, overriding the
                                                                   #  cached versions if they exist
                                             )
# Set PT model to inference mode
model.eval()
# Build HF Transformers pipeline
pipeline = transformers.pipeline( # connect the llm and tokenizer
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    device_map="auto",
    max_length=400, #
    do_sample=True,
    top_k=10,
    num_return_sequences=1,
    eos_token_id=tokenizer.eos_token_id
)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/287 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.73M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/281 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.05k [00:00<?, ?B/s]

...

pytorch_model.bin.index.json:   0%|          | 0.00/16.9k [00:00<?, ?B/s]

pytorch_model-00001-of-00002.bin:   0%|          | 0.00/9.95G [00:00<?, ?B/s]

pytorch_model-00002-of-00002.bin:   0%|          | 0.00/4.48G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/117 [00:00<?, ?B/s]

In [29]:
# Test out the pipeline
pipeline('what is the independence day of india?')

Setting `pad_token_id` to `eos_token_id`:11 for open-end generation.


[{'generated_text': 'what is the independence day of india?\nIndependence day of India is celebrated on August 15th every year.'}]

In [31]:

pipeline('what Occasion in india is on 15th august?')

Setting `pad_token_id` to `eos_token_id`:11 for open-end generation.


[{'generated_text': "what Occasion in india is on 15th august?\nI'm sorry, but I could not find information on a specific event on August 15th in India."}]

In [34]:

pipeline('what Occasion in India is on August 15th ?')

Setting `pad_token_id` to `eos_token_id`:11 for open-end generation.


[{'generated_text': "what Occasion in India is on August 15th ?\nI'm sorry, but as an AI language model, I am not currently programmed to provide specific event information. However, there are several websites and online resources that offer calendars and lists of events in India, including the one at the bottom of the article."}]

In [8]:

# Setup prompt template
template = PromptTemplate(input_variables=['input'], template='{input}')
# Pass hugging face pipeline to langchain class
llm = HuggingFacePipeline(pipeline=pipeline)
# Build stacked LLM chain i.e. prompt-formatting + LLM
chain = LLMChain(llm=llm, prompt=template)

In [18]:
# Test LLMChain
response = chain.run('what is intutuiion of P-value in statisctiacl analysis?')

Setting `pad_token_id` to `eos_token_id`:11 for open-end generation.


In [19]:
response

'what is intutuiion of P-value in statisctiacl analysis?\nIn statistics, a P-value is a measure of evidence to reject a null hypothesis (i.e. an assertion or statement about an observed phenomenon that has no alternative explanation other than the null hypothesis). The value of a P-value indicates the level of significance at which a particular observation was made, and can range from 0 (strong evidence of a difference) to 1 (strong evidence of a difference in a larger sample size). In statistical analysis, the P-value is used to determine the strength of evidence against a null hypothesis.'